In [23]:
#from aicsimageio.writers import OmeTiffWriter
#from aicsimageio import aics_image as AI
#from aicsimageio import AICSImage, imread, ome_tiff_writer
#import dask
#import dask.array as da
import os
import os
import numpy as np
import tifffile as tiff

In [20]:
### save with AICSImage -- not correct even though it looks correct in imagej

# Paths to input and output directories
input_dir = './../../../../data/Human_squamous_cell_carcinoma_stained_with_SignalStar_mIHC_technology/test/'
output_file = './../../../../data/Human_squamous_cell_carcinoma_stained_with_SignalStar_mIHC_technology/stacked_output.ome.tiff'

# Make list of all files with a ".tif" extension in the input directory
input_files = [os.path.join(input_dir, file) for file in os.listdir(input_dir) if file.endswith('.tif')]
# Sort the list of files alphabetically
input_files = sorted(input_files)

# Load the first image to determine the shape
first_img = AICSImage(input_files[0]).get_image_dask_data("CZYX")

# Number of tiles (one file per tile)
num_tiles = len(input_files)
# Number of channels in each tile (assumed to be the same for all tiles)
num_channels = first_img.shape[0]

# Initialize an empty dask array to hold all images
# Shape: (Tiles, Channels, Z, Y, X)
stack_shape = (num_channels, num_tiles, first_img.shape[1], first_img.shape[2], first_img.shape[3])
out = da.empty(stack_shape, dtype=first_img.dtype)

# Loop through each file and stack the images
for idx, file in enumerate(input_files):
    img = AICSImage(file).get_image_dask_data("CZYX")
    for channel in range(num_channels):
        out[channel, idx] = img[channel]

# Reshape the array to match the required order TCZYX (Tiles as time points)
out = out.reshape((num_channels, num_tiles, first_img.shape[1], first_img.shape[2], first_img.shape[3]))

# Save the stacked image using OmeTiffWriter
OmeTiffWriter.save(out, output_file, dim_order="CTZYX")

print(f"Stacked image saved to {output_file}")


Stacked image saved to ./../../../../data/Human_squamous_cell_carcinoma_stained_with_SignalStar_mIHC_technology/stacked_output.ome.tiff


In [26]:
# try to save as numpy array: WORKS

# Paths to input and output directories
input_dir = './../../../../data/Human_squamous_cell_carcinoma_stained_with_SignalStar_mIHC_technology/test/'
output_file = './../../../../data/Human_squamous_cell_carcinoma_stained_with_SignalStar_mIHC_technology/numpy_stacked_output.tif'

# Make list of all files with a ".tif" extension in the input directory
input_files = [os.path.join(input_dir, file) for file in os.listdir(input_dir) if file.endswith('.tif')]
# Sort the list of files alphabetically
input_files = sorted(input_files)

# Load the first image to determine the shape
first_img = np.asarray(tiff.imread(input_files[0]))

# Number of tiles (one file per tile)
num_tiles = len(input_files)
# Number of channels in each tile (assumed to be the same for all tiles)
num_channels = first_img.shape[0]

# Initialize an empty numpy array to hold all images
# Shape: (Tiles, Channels, Z, Y, X)
stack_shape = (num_tiles, num_channels, first_img.shape[1], first_img.shape[2])
stacked_image = np.empty(stack_shape, dtype=first_img.dtype)

# Loop through each file and stack the images
for idx, file in enumerate(input_files):
    img = np.asarray(tiff.imread(file))
    stacked_image[idx] = img

# Save the stacked image as a TIFF file
tiff.imsave(output_file, stacked_image)

print(f"Stacked image saved to {output_file}")


Stacked image saved to ./../../../../data/Human_squamous_cell_carcinoma_stained_with_SignalStar_mIHC_technology/numpy_stacked_output.tif


/var/folders/79/cjwcycvd2_9dyfffbym60x200000gn/T/ipykernel_14159/2789077634.py:31: DeprecationWarning: <tifffile.imsave> is deprecated. Use tifffile.imwrite
  tiff.imsave(output_file, stacked_image)


In [27]:
#convert the file to uint16

# Read the float TIFF
img = tiff.imread('./../../../../data/Human_squamous_cell_carcinoma_stained_with_SignalStar_mIHC_technology/numpy_stacked_output.tif')

# Convert the image to uint16 (scale if necessary)
img_uint16 = (img * 65535).astype(np.uint16)

# Save the converted image
tiff.imwrite('./../../../../data/Human_squamous_cell_carcinoma_stained_with_SignalStar_mIHC_technology/numpy_stacked_output_uint16.tif', img_uint16)